In [64]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from alu import ArithmeticAttentionModel

In [74]:
model = ArithmeticAttentionModel()

In [75]:
num1 = torch.tensor([[1]]).float()
num2 = torch.tensor([[2]]).float()
op = torch.tensor([[0, 0, 1, 0]]).float()

In [76]:
model(num1, num2, op)

tensor([[37456.8516]], grad_fn=<SumBackward1>)

In [77]:
def arithmetic_loss(predictions, targets, scale_factor=10000.0):
    abs_error = (predictions - targets)**2
    # rel_error = torch.abs((predictions - targets) / (targets + 1e-8)) * scale_factor
    loss = abs_error # + rel_error
    return torch.sum(loss)

In [78]:
class ArithmeticDataGenerator:
    def __init__(self, min_val=0, max_val=512, device='cuda'):
        self.min_val = min_val
        self.max_val = max_val
        self.device = device
        self.operations = {
            0: lambda x, y: x + y,    # addition
            1: lambda x, y: x - y,    # subtraction
            2: lambda x, y: x * y,    # multiplication
            3: lambda x, y: x / (y + 1e-8)  # division
        }
    
    def generate_batch(self, batch_size):
        # Generate random numbers
        num1 = torch.rand(batch_size, device=self.device) * (self.max_val - self.min_val) + self.min_val
        num2 = torch.rand(batch_size, device=self.device) * (self.max_val - self.min_val) + self.min_val
        
        # Generate random operations
        op_idx = torch.randint(0, 1, (batch_size,), device=self.device)
        operation = F.one_hot(op_idx, num_classes=4).float()
        
        # Calculate targets
        targets = torch.zeros(batch_size, 1, device=self.device)
        for i, op in enumerate(op_idx):
            targets[i] = self.operations[op.item()](num1[i], num2[i])
        
        return num1.view(-1, 1), num2.view(-1, 1), operation, targets


In [79]:
from tqdm import tqdm
import numpy as np

def train_model(
    model,
    num_epochs=100,
    batch_size=64,
    learning_rate=1e-4,
    device='cuda',
    eval_every=100
):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    data_gen = ArithmeticDataGenerator(device=device)
    
    steps_per_epoch = 1000
    best_loss = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        epoch_losses = []
        
        # Progress bar for each epoch
        pbar = tqdm(range(steps_per_epoch), desc=f'Epoch {epoch+1}/{num_epochs}')
        
        for step in pbar:
            num1, num2, operation, targets = data_gen.generate_batch(batch_size)
            
            optimizer.zero_grad()
            predictions = model(num1, num2, operation)
            loss = arithmetic_loss(predictions, targets)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            epoch_losses.append(loss.item())
            
            # Update progress bar
            if step % eval_every == 0:
                # Evaluate on a test batch
                with torch.no_grad():
                    model.eval()
                    test_num1, test_num2, test_op, test_targets = data_gen.generate_batch(batch_size)
                    test_pred = model(test_num1, test_num2, test_op)
                    test_loss = arithmetic_loss(test_pred, test_targets)
                    
                    # Sample prediction
                    idx = 0
                    op_name = ['add', 'subtract', 'multiply', 'divide'][test_op[idx].argmax().item()]
                    print( #pbar.set_description(
                        f'Epoch {epoch+1}/{num_epochs} | '
                        f'Loss: {loss.item():.4f} | '
                        f'Test Loss: {test_loss.item():.4f} | '
                        f'Sample: {test_num1[idx].item():.2f} {op_name} {test_num2[idx].item():.2f} = '
                        f'{test_pred[idx].item():.2f} (true: {test_targets[idx].item():.2f})'
                    )
                model.train()
            
        # End of epoch
        avg_loss = np.mean(epoch_losses)
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(model.state_dict(), 'best_arithmetic_model.pt')
            
        scheduler.step()
        
        print(f'\nEpoch {epoch+1} completed. Average loss: {avg_loss:.4f}')


In [ ]:
train_model(model, num_epochs=10, batch_size=1024, learning_rate=1e-4, device='cuda', eval_every=100)

Epoch 1/10:   0%|▍                                                                                | 5/1000 [00:00<00:40, 24.58it/s]

Epoch 1/10 | Loss: 525481.0625 | Test Loss: 278102432.0000 | Sample: 342.24 add 436.77 = 21.18 (true: 779.02)


Epoch 1/10:  10%|████████▏                                                                      | 104/1000 [00:03<00:33, 26.42it/s]

Epoch 1/10 | Loss: 38304568.0000 | Test Loss: 37568072.0000 | Sample: 496.06 add 487.82 = 654.38 (true: 983.88)


Epoch 1/10:  21%|████████████████▎                                                              | 206/1000 [00:07<00:30, 26.43it/s]

...

Epoch 7/10:  30%|████████████████████████                                                       | 305/1000 [00:11<00:26, 26.35it/s]

Epoch 7/10 | Loss: 80887.2891 | Test Loss: 66881.6641 | Sample: 128.10 add 241.41 = 375.21 (true: 369.50)


Epoch 7/10:  40%|███████████████████████████████▉                                               | 404/1000 [00:14<00:22, 26.19it/s]

Epoch 7/10 | Loss: 176956.7188 | Test Loss: 103812.0312 | Sample: 443.49 add 70.74 = 517.35 (true: 514.23)


Epoch 7/10:  44%|██████████████████████████████████▌                                            | 437/1000 [00:15<00:20, 27.84it/s]